In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "dlyfrzthw"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_dlyfrzthw_monthly.nc

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [7]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    dfp=0
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"], 2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"], 2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    Region1i45=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF 

In [8]:
df45list


[                      january_hist_rcp45_p10  january_t2050_rcp45_p10  \
 lat       lon                                                           
 40.041039 -89.045212                    9.97                    10.57   
           -88.961884                    9.93                    10.60   
           -88.878548                    9.90                    10.73   
           -88.795219                    9.77                    10.60   
           -88.711891                    9.83                    10.57   
 ...                                      ...                      ...   
 66.623306 -61.629639                    0.00                     0.00   
           -61.546310                    0.00                     0.00   
           -61.462978                    0.00                     0.00   
           -61.379650                    0.00                     0.00   
           -61.296322                    0.00                     0.00   
 
                       january_t2080

In [9]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [10]:
mkey = ["lat", "lon"]

In [11]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [12]:
dfse452 = dfse45.reset_index()
dfse45=0

In [13]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [14]:
dfse452 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [15]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


,lat,lon,january_t2050_rcp45_p10,january_t2080_rcp45_p10,january_t2050_rcp45_p50,january_t2080_rcp45_p50,january_t2050_rcp45_p90,january_t2080_rcp45_p90,february_t2050_rcp45_p10,february_t2080_rcp45_p10,...,november_t2050_rcp45_p50,november_t2080_rcp45_p50,november_t2050_rcp45_p90,november_t2080_rcp45_p90,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,10.57,10.63,15.67,15.33,21.10,19.87,10.60,10.13,...,9.10,8.23,14.03,13.70,11.13,10.03,15.53,14.90,20.43,20.20
1,40.041039,-88.961884,10.60,10.70,15.70,15.40,21.10,19.97,10.63,10.27,...,8.97,8.17,14.07,13.67,11.23,10.00,15.57,15.10,20.60,20.10
2,40.041039,-88.878548,10.73,10.60,15.83,15.53,21.10,20.03,10.63,10.20,...,8.97,8.07,13.97,13.60,11.27,10.07,15.77,15.23,20.60,20.10
3,40.041039,-88.795219,10.60,10.70,15.63,15.40,21.03,19.93,10.23,9.97,...,8.83,7.90,13.70,13.40,11.07,10.00,15.37,14.93,20.40,19.83
4,40.041039,-88.711891,10.57,10.67,15.73,15.30,21.07,19.87,10.20,9.97,...,8.87,7.83,13.63,13.13,10.90,9.90,15.47,14.90,20.23,19.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,1.03,0.97,0.00,0.00,...,0.00,0.03,2.00,1.83,0.00,0.00,0.00,0.00,2.00,2.87
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.83,0.77,0.00,0.00,...,0.00,0.00,1.37,1.17,0.00,0.00,0.00,0.00,1.90,2.80
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.87,0.90,0.00,0.00,...,0.00,0.00,1.63,1.57,0.00,0.00,0.00,0.00,2.07,2.83
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.93,0.90,0.00,0.00,...,0.00,0.00,1.87,1.70,0.00,0.00,0.00,0.00,2.07,2.90


In [16]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [17]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [18]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    dfp852=0
    df85C[var+"_p10"] = round(df85C[var+"_p10"], 2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"], 2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    Region1i85=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF added to the list
Dataset number:  5
Percentiles Obteined
Converted to DF
S

In [19]:
df85list


[                      january_hist_rcp85_p10  january_t2050_rcp85_p10  \
 lat       lon                                                           
 40.041039 -89.045212                   10.20                    10.73   
           -88.961884                   10.10                    10.77   
           -88.878548                   10.27                    10.77   
           -88.795219                   10.17                    10.57   
           -88.711891                   10.07                    10.53   
 ...                                      ...                      ...   
 66.623306 -61.629639                    0.00                     0.00   
           -61.546310                    0.00                     0.00   
           -61.462978                    0.00                     0.00   
           -61.379650                    0.00                     0.00   
           -61.296322                    0.00                     0.00   
 
                       january_t2080

In [20]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [21]:
dfse85

,lat,lon,january_hist_rcp85_p10,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_hist_rcp85_p50,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_hist_rcp85_p90,january_t2050_rcp85_p90,...,november_t2080_rcp85_p90,december_hist_rcp85_p10,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_hist_rcp85_p50,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_hist_rcp85_p90,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,10.20,10.73,8.63,15.93,15.53,14.57,20.93,20.20,...,9.23,13.70,9.50,6.53,18.53,14.27,11.97,23.47,18.90,16.53
1,40.041039,-88.961884,10.10,10.77,8.70,16.13,15.57,14.67,21.00,20.17,...,9.23,13.67,9.53,6.53,18.53,14.47,12.13,23.40,18.93,16.67
2,40.041039,-88.878548,10.27,10.77,8.77,16.17,15.67,14.93,21.07,20.20,...,9.27,13.63,9.70,6.47,18.33,14.57,12.03,23.40,18.87,16.67
3,40.041039,-88.795219,10.17,10.57,8.70,15.90,15.53,14.90,20.90,20.13,...,9.07,13.40,9.57,6.43,18.07,14.40,11.90,23.00,18.73,16.57
4,40.041039,-88.711891,10.07,10.53,8.70,15.90,15.50,14.83,20.87,20.13,...,9.03,13.37,9.50,6.30,18.17,14.33,11.73,23.00,18.70,16.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,0.00,0.00,0.17,1.67,...,12.43,0.00,0.00,0.00,0.00,0.00,0.00,0.30,5.17,4.43
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,0.00,0.00,0.03,1.47,...,8.77,0.00,0.00,0.00,0.00,0.00,0.00,0.20,4.43,4.23
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,0.00,0.00,0.13,1.60,...,10.33,0.00,0.00,0.00,0.00,0.00,0.00,0.27,4.77,4.17
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,0.00,0.00,0.17,1.63,...,10.63,0.00,0.00,0.00,0.00,0.00,0.00,0.27,4.97,4.20


In [22]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [23]:
dfse85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [24]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2050_rcp85_p50,november_t2080_rcp85_p50,november_t2050_rcp85_p90,november_t2080_rcp85_p90,december_t2050_rcp85_p10,december_t2080_rcp85_p10,december_t2050_rcp85_p50,december_t2080_rcp85_p50,december_t2050_rcp85_p90,december_t2080_rcp85_p90
0,40.041039,-89.045212,10.73,8.63,15.53,14.57,20.20,19.13,9.97,7.47,...,7.77,4.37,12.13,9.23,9.50,6.53,14.27,11.97,18.90,16.53
1,40.041039,-88.961884,10.77,8.70,15.57,14.67,20.17,19.37,10.07,7.63,...,7.73,4.40,12.13,9.23,9.53,6.53,14.47,12.13,18.93,16.67
2,40.041039,-88.878548,10.77,8.77,15.67,14.93,20.20,19.53,10.17,7.70,...,7.57,4.33,12.03,9.27,9.70,6.47,14.57,12.03,18.87,16.67
3,40.041039,-88.795219,10.57,8.70,15.53,14.90,20.13,19.50,10.20,7.57,...,7.43,4.30,11.87,9.07,9.57,6.43,14.40,11.90,18.73,16.57
4,40.041039,-88.711891,10.53,8.70,15.50,14.83,20.13,19.33,10.10,7.53,...,7.20,4.10,11.77,9.03,9.50,6.30,14.33,11.73,18.70,16.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,1.67,7.47,0.00,0.00,...,0.03,0.40,3.00,12.43,0.00,0.00,0.00,0.00,5.17,4.43
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,1.47,6.80,0.00,0.00,...,0.00,0.07,2.03,8.77,0.00,0.00,0.00,0.00,4.43,4.23
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,1.60,7.13,0.00,0.00,...,0.03,0.17,2.60,10.33,0.00,0.00,0.00,0.00,4.77,4.17
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,1.63,7.20,0.00,0.00,...,0.03,0.23,2.73,10.63,0.00,0.00,0.00,0.00,4.97,4.20


In [25]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

january
february
march
april
may
june
july
august
september
october
november
december


In [26]:
lisdfmoh85

[             lat        lon  january_hist_rcp85_p50
 0      40.041039 -89.045212                   15.93
 1      40.041039 -88.961884                   16.13
 2      40.041039 -88.878548                   16.17
 3      40.041039 -88.795219                   15.90
 4      40.041039 -88.711891                   15.90
 ...          ...        ...                     ...
 76201  66.623306 -61.629639                    0.00
 76202  66.623306 -61.546310                    0.00
 76203  66.623306 -61.462978                    0.00
 76204  66.623306 -61.379650                    0.00
 76205  66.623306 -61.296322                    0.00
 
 [76206 rows x 3 columns],              lat        lon  february_hist_rcp85_p50
 0      40.041039 -89.045212                    16.30
 1      40.041039 -88.961884                    16.43
 2      40.041039 -88.878548                    16.33
 3      40.041039 -88.795219                    16.27
 4      40.041039 -88.711891                    16.07
 ...        

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [27]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfw45=0
    dfw85=0
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dfsew=0
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    dftgwh2=0
    return (dftgwh3)


### All months historic mean

In [28]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

january
january
february
february
march
march
april
april
may
may
june
june
july
july
august
august
september
september
october
october
november
november
december
december


# Merge means with the 45 and 85 DF

In [29]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [30]:
dftgseall

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,10.73,8.63,15.53,14.57,20.20,19.13,9.97,7.47,...,13.70,13.70,11.13,10.03,18.67,15.53,14.90,23.40,20.43,20.20
1,40.041039,-88.961884,10.77,8.70,15.57,14.67,20.17,19.37,10.07,7.63,...,13.67,13.70,11.23,10.00,18.60,15.57,15.10,23.27,20.60,20.10
2,40.041039,-88.878548,10.77,8.77,15.67,14.93,20.20,19.53,10.17,7.70,...,13.60,13.63,11.27,10.07,18.47,15.77,15.23,23.23,20.60,20.10
3,40.041039,-88.795219,10.57,8.70,15.53,14.90,20.13,19.50,10.20,7.57,...,13.40,13.37,11.07,10.00,18.30,15.37,14.93,22.93,20.40,19.83
4,40.041039,-88.711891,10.53,8.70,15.50,14.83,20.13,19.33,10.10,7.53,...,13.13,13.50,10.90,9.90,18.23,15.47,14.90,22.93,20.23,19.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,1.67,7.47,0.00,0.00,...,1.83,0.00,0.00,0.00,0.00,0.00,0.00,0.27,2.00,2.87
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,1.47,6.80,0.00,0.00,...,1.17,0.00,0.00,0.00,0.00,0.00,0.00,0.23,1.90,2.80
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,1.60,7.13,0.00,0.00,...,1.57,0.00,0.00,0.00,0.00,0.00,0.00,0.30,2.07,2.83
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,1.63,7.20,0.00,0.00,...,1.70,0.00,0.00,0.00,0.00,0.00,0.00,0.30,2.07,2.90


In [31]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [32]:
dfmop50

,lat,lon,january_hist_p50,february_hist_p50,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,15.75,16.35,16.60,3.86,0.00,0.00,0.00,0.00,0.00,3.54,12.82,18.60
1,40.041039,-88.961884,15.88,16.42,16.57,3.96,0.00,0.00,0.00,0.00,0.00,3.54,12.84,18.57
2,40.041039,-88.878548,15.97,16.30,16.62,4.02,0.00,0.00,0.00,0.00,0.00,3.52,12.80,18.40
3,40.041039,-88.795219,15.78,16.17,16.48,3.95,0.00,0.00,0.00,0.00,0.00,3.42,12.46,18.19
4,40.041039,-88.711891,15.75,16.07,16.48,3.95,0.00,0.00,0.00,0.00,0.00,3.25,12.22,18.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.20,8.70,17.11,2.25,4.86,15.40,3.30,0.00,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.07,7.52,17.67,2.90,5.75,14.85,2.43,0.00,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.15,8.20,17.37,2.55,5.10,15.20,3.04,0.00,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.20,8.55,17.35,2.37,4.95,15.47,3.32,0.00,0.00


In [33]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [34]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [35]:
dftgseall2

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,november_t2080_rcp45_p90,december_hist_rcp45_p10,december_t2050_rcp45_p10,december_t2080_rcp45_p10,december_hist_rcp45_p50,december_t2050_rcp45_p50,december_t2080_rcp45_p50,december_hist_rcp45_p90,december_t2050_rcp45_p90,december_t2080_rcp45_p90
0,40.041039,-89.045212,10.73,8.63,15.53,14.57,20.20,19.13,9.97,7.47,...,13.70,13.70,11.13,10.03,18.67,15.53,14.90,23.40,20.43,20.20
1,40.041039,-88.961884,10.77,8.70,15.57,14.67,20.17,19.37,10.07,7.63,...,13.67,13.70,11.23,10.00,18.60,15.57,15.10,23.27,20.60,20.10
2,40.041039,-88.878548,10.77,8.77,15.67,14.93,20.20,19.53,10.17,7.70,...,13.60,13.63,11.27,10.07,18.47,15.77,15.23,23.23,20.60,20.10
3,40.041039,-88.795219,10.57,8.70,15.53,14.90,20.13,19.50,10.20,7.57,...,13.40,13.37,11.07,10.00,18.30,15.37,14.93,22.93,20.40,19.83
4,40.041039,-88.711891,10.53,8.70,15.50,14.83,20.13,19.33,10.10,7.53,...,13.13,13.50,10.90,9.90,18.23,15.47,14.90,22.93,20.23,19.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,1.67,7.47,0.00,0.00,...,1.83,0.00,0.00,0.00,0.00,0.00,0.00,0.27,2.00,2.87
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,1.47,6.80,0.00,0.00,...,1.17,0.00,0.00,0.00,0.00,0.00,0.00,0.23,1.90,2.80
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,1.60,7.13,0.00,0.00,...,1.57,0.00,0.00,0.00,0.00,0.00,0.00,0.30,2.07,2.83
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,1.63,7.20,0.00,0.00,...,1.70,0.00,0.00,0.00,0.00,0.00,0.00,0.30,2.07,2.90


In [36]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)
dftgseall2=0

In [37]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [38]:
dftgseall3

,lat,lon,january_t2050_rcp85_p10,january_t2080_rcp85_p10,january_t2050_rcp85_p50,january_t2080_rcp85_p50,january_t2050_rcp85_p90,january_t2080_rcp85_p90,february_t2050_rcp85_p10,february_t2080_rcp85_p10,...,march_hist_p50,april_hist_p50,may_hist_p50,june_hist_p50,july_hist_p50,august_hist_p50,september_hist_p50,october_hist_p50,november_hist_p50,december_hist_p50
0,40.041039,-89.045212,10.73,8.63,15.53,14.57,20.20,19.13,9.97,7.47,...,16.60,3.86,0.00,0.00,0.00,0.00,0.00,3.54,12.82,18.60
1,40.041039,-88.961884,10.77,8.70,15.57,14.67,20.17,19.37,10.07,7.63,...,16.57,3.96,0.00,0.00,0.00,0.00,0.00,3.54,12.84,18.57
2,40.041039,-88.878548,10.77,8.77,15.67,14.93,20.20,19.53,10.17,7.70,...,16.62,4.02,0.00,0.00,0.00,0.00,0.00,3.52,12.80,18.40
3,40.041039,-88.795219,10.57,8.70,15.53,14.90,20.13,19.50,10.20,7.57,...,16.48,3.95,0.00,0.00,0.00,0.00,0.00,3.42,12.46,18.19
4,40.041039,-88.711891,10.53,8.70,15.50,14.83,20.13,19.33,10.10,7.53,...,16.48,3.95,0.00,0.00,0.00,0.00,0.00,3.25,12.22,18.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,0.00,0.00,0.00,0.00,1.67,7.47,0.00,0.00,...,0.00,0.20,8.70,17.11,2.25,4.86,15.40,3.30,0.00,0.00
76202,66.623306,-61.546310,0.00,0.00,0.00,0.00,1.47,6.80,0.00,0.00,...,0.00,0.07,7.52,17.67,2.90,5.75,14.85,2.43,0.00,0.00
76203,66.623306,-61.462978,0.00,0.00,0.00,0.00,1.60,7.13,0.00,0.00,...,0.00,0.15,8.20,17.37,2.55,5.10,15.20,3.04,0.00,0.00
76204,66.623306,-61.379650,0.00,0.00,0.00,0.00,1.63,7.20,0.00,0.00,...,0.00,0.20,8.55,17.35,2.37,4.95,15.47,3.32,0.00,0.00


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [39]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [40]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [41]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [42]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    dfpolyshape = 0
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")
    dftp = 0
    gdf = 0

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_dlyfrzthw_monthly.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
